In [ ]:
import marimo as mo

# CIFAR Demonstration

This notebook demonstrates how to use the `hierarchicalsoftmax` module to train a neural network on the [CIFAR](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

First, choose the hyperparameters.

In [ ]:
cifar_radio = mo.ui.radio(options=["10","100"], value=mo.cli_args().get("cifar") or "100", label="CIFAR Dataset")
batch_size_input = mo.ui.number(value=mo.cli_args().get("batch") or 32, label="Batch Size")
epochs_input = mo.ui.number(value=mo.cli_args().get("batch") or 10, label="Epochs")
mo.vstack([cifar_radio, epochs_input, batch_size_input])

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

assert cifar_radio.value in ["10","100"]
batch_size = batch_size_input.value
epochs = epochs_input.value
cifar_dataset = datasets.CIFAR10 if cifar_radio.value == "10" else datasets.CIFAR100

# Use the same data augmentation strategies as in https://arxiv.org/pdf/1605.07146v4
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4, padding_mode="reflect"),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_data = cifar_dataset(root=".", train=True, download=True, transform=transform)
test_data = cifar_dataset(root=".", train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

### Plot the first 10 images

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

num_images = 10

# Create a row of subplots
cifar_fig = make_subplots(
    rows=1, cols=num_images, 
    subplot_titles=[train_data.classes[train_data[i][1]] for i in range(num_images)], 
    horizontal_spacing=0,
)

for i in range(num_images):
    img, label = train_data[i]
    img = img.permute(1, 2, 0).numpy()  # (C, H, W) -> (H, W, C) and convert to numpy

    cifar_fig.add_trace(
        go.Image(z=(img * 255).astype('uint8')),
        row=1, col=i+1
    )

# Update layout: remove axes and tighten spacing
thumbnail_size = 105
cifar_fig.update_layout(
    height=thumbnail_size,  # adjust height as needed
    width=thumbnail_size * num_images,  # 150px per image
    showlegend=False,
    margin=dict(l=0, r=0, t=30, b=0)
)

# Hide axes
for i in range(1, num_images + 1):
    cifar_fig.update_xaxes(visible=False, row=1, col=i)
    cifar_fig.update_yaxes(visible=False, row=1, col=i)

cifar_fig

<marimo-plotly data-figure='{"data": [{"z": {"dtype": "u1", "bdata": "/////v7+/////////////////////////////////////////////////////v///vz3/vPa2b+eelhLkIKD6Ofo6+jq+vj5//7+/f7/9vv83OjewdC5vcepprKCeYVX/////v7+/////////////////////f///P34+Ozk793Y7uPj/P79/P///f79//z58ePa0beisI52fFpRgmllqJ2Vubey8/Lv+vz95+zv4ODX1NjBwMysp7WHlaJueINZ/////v7+/////////////////////f386NnQ0qaUyJSFsI6H1svH+/r7/v//0tHMoZCLlHduhmFXgV5YelZLZ1JGkJGN8vX18vX31tbQtKGGnYdjsKaBnaR2kJhyfoVv/////v7+/////////////////////PX11bKt0paG2peByJB7tI2C2srI4uTmdH1/SUlPa1pafVxTiGFXfllQRTAxKyw5qa245uPhwa6WhVs1fkskkmxOnJaBmp+Vnaam/////v/+/////////////////////vv74cjHzJaMyoJq04houoBpspKMqKOrLjdEJCo3VktQeFxYfVpUVTs4LSUwDxIoTk1XjX1tg2A5jlgqjFQri2FEsaKVs7m5q7q//////v7+//////////7//////////f//+fX23Ly3uXhlwHBQtHdZjGxjgXiDMjJDFhclRz1CgGppcllYJxwgEhYmHCE1MCkoVjsge08liVQkeU0jhmpNmIh4pqynrL66//////3+/v///v////3+//7+/////v3+/vz+9/Ly3MG3w4980KmZoZWWXl5lOzY6KiQrh3p9vqmnUT48GhMaFxooGx0oJh4eXUYxeU4pcj4bdk8yh3ViaV1VWlVQc3dy/f3+//3+/P7+9Pj3+vz7/f79/////v7/+v3+1ODew8i/vLWo7ujiy8rQS0pSNi0ve2xr2cbDtKKhOzM5FBIdGRklISEoMykmaFZGalE7dVVCdV5Qa2BVZlRMVEQ+Mikk9vj4/v7+7vTys7+7wcvH8vb1////+/v73+bhlKSac4Z8bH12w8/Iv73ANSszKR4jnoyI3cbApJOTNTE8EA8bEg8YGxgdNikjVUQ1Pi4hRzQsUkM7XlNGa1dJXUg9Tz009/z78PLyucLAlaagnKyn1t3b/f3+7O3or7ale4RwW2ZcRVVQk6KOyb62V0NIRj1FraChwKyoi3p5KiQrIBskFQ8XDQcLPS4nYUw5SzclSDMoTzgqW0UtZVZAXUw6Xkw99PX4tbu8gpCOpbiqi5ubhpGY0tvWr7utjJmEfYd4SVBIKjQom5+F9t7Wv6Wlr5aX0rCwuJaVZk1PHREWLCQpHRcfIA0XWD80YE4zTDUjSzUkUjslV0AlW0owXk42W0w25+npc314hpeMnrCibHmDbXuCnK2bcIJtcYJxdIJ1SVRBandYsbCX8tTQ68zM7bu70XR4wWBoq1plfjdESiIrIiAkQjM4bUc8ZUYvSDIhRjAcTjkhWUQqVUAjW0ktXU407PTwgo+Ea3xpWGhgOURUUmBdgJN0boBpXW5hYnBgaXZXgpFlqKmS7MzK6MPE4pugx1RfskVOtFVds1Vglz5RWCs3XERDa0w/ZEk0STAcQysUTzccWUIkVj4gV0IlXUswtcPHf46Hbn9tN0Y/HCkzRVVIfZBuYXNfTFxTbXpvg492Z3VQk5WC8dTT8L3By2t1nEZUURsjQhkbUiElmEhZkkRUdS00eDw5Z0Y1RCkUQicOTjUYVTscUjgXVT0eW0YplKOag5KGhJKBOEY9JDI2iZePscGxeol/d4V+lJ6cqbCppaydw8G39NbT5aOiuF1fekZPXklRTUBDQi40SSkvdD9IlS47lSQybzMvQyUNPiEHTDATUjUXSC0LUDgUW0YlrsORpbeQoLKKf5FybH5pjZ+Mq7urq7qqo6+jiZGPxMfH5+fm19PY276+yYaGuoF+t56fr6CkoZSbnZGbcmpuVkNKijE6mCYwbjEuRSYLPx8DSywQTi8TQigGTDUMV0IccI5IbIpFiqpar8uEkKhskqlwma95l6x8ip14doNqjpiAnKSNgYl/iYJ3sYmKs5GYm4+WYFpiXFhiYV9rUlNWTEhNezk/jjQzZDQoQiQIQiMHTzAUSiwQRCkITzkPTzsVk7Jbjq5VpMZfob9lg51Th6FajaZifplUeZVTiJ9ok6Zxn651oLF3nq55vrKXgXZvVlFUY15kYWBoWVpkP0NEPUJIekpRgTkuWzMeQyUJSSoOVTYaRigMSC4NUTsRTTkSx+KCu9Z5kaxKf5w5p8ZltNJztNJ0nr1hmbdijKdbhZxTprhrucp8rcJ+xsWnhYB8U0xLaV9gc2xvdXB1SUVHWV5oi2RsfDgrWTQYSiwPTC0QUTIVQSMGTDEQTjcPWEMeuNZuutZ1g51IV3QkW3omd5g6pchdvNtxxd1/xNqAwdZ9x9uGyOCGw9eF3+HKtrCxmY+OXlFTWk5QVkpNNCoxSFBaiHZ9fDUsXTwVUTgUSC0KSi8MRysITzEOSS4LWD8ewtxwwdpzvtV7psBxhKFRbow1lbVRrc1kpcRctdRqwuF3xuZ+uthvzOKL9Pjf0s3TgXd4S0E/ST89ST89NTIxWmJimpGNfTcsYj0XV0EaUTkTSDAKRi0HSy8KRywKTjYVxN9sxuFvz+mA0u2Mz+yLx+SCxeJ/rM5mj7VIj7ZFp89att5itNpWwN1x4fCpt7ubVU09RDkpTEExTEExQEEwd3Rmi4Rvf0c0ckwrYEwlWUMcTTgQQSsESS8KRy8MSzcVnsVIrdVVtNthwuNs0/B5z+x70eyBwt95sdJvo8Vaqs5bqM5Skbk/gqI3nLZCuMVzsKN7g25MfWhGjXlXhn5ghW5XX1o4f1xAhWdGblw0W0khW0ggTDkRQCoESDQPVUUhjrk6lsFAnchGr9VVtddgtddev+Bpy+mCyuWFxuF9wdx0ss9mdZU4eZc7boslf5NCpZlrlXtbr5F1sY9xqZlyloNcem5FiHJUkX1hgnNOaVkvY1EoVkIcQi0ITzscZlY5qtBepsxcqM9epsxYmbxVi69AqM5YtdVvrchkudRwzOeDxuF+l7RUocBcf50+hqFNp7pppbBpo6RopqFhycyFp6RvkX9kmYZulohslYlrhnhRdWQ+ZE8vTDkXU0IjaFk9pMxTq9Ncttxus9hqnL1glrlYpcleo8Req8piutlwyumAvNtyrcxit9ZsudhutdVrrtNiweV2uNZtwNt0zOV8r7d3mox9nIp2loprl410j4NhfG1LaVc8STgaUUMeX1UvmcJBoMpAnsdFochPnb5hpcZqm79ToMNYrM5hr9Fkrc9iocNVocFUrMpjr89krc1Zr81avttnvNlltdFms9VhsMZsm5dukYNojIJliIJkgXlTd2xGTT4hMSIIRjwRUEsalLlPjrY5jLM8hq02jK9Me51CbZInmb9NkbZFk7dJkbVHkbNIcJAtd5Y3h6hAlrlCpclIr9VPt99Zps1RnMdLm71Hk5xMjINain1ig4BYfH1FbWczKyAEHhEBQT4PTE0Uep1CeJs6fqBHY4cpXYEkYYUtcZY1fqY2gag3g6hAja1MlbNYdJdDT3MbaY4pbZQicpkgfqMtia48gKU0g6sygKokjKRAiYVRiXRRcnA6c4YtW1saHhQCFhADYXA4jaFXV3opWHonZYY4VXUrSGgecpQ+krVRdpsxg6hCgqNBl7NanrhmlLVZhqlLfqFBbpEyY4MwdJVAfZ5Df6BEdJowbpcZcIgrfnpKhXNManIvcYwvTE4YEwkEIiQKaYU7iq1Pep1CeJs6fqBHY4cpXYEkYYUtcZY1fqY2gag3g6hAja1MlbNYdJdDT3MbaY4pbZQicpkgfqMtia48gKU0g6sygKokjK

## Non-hierarchical model

First we create a basic non-hierarchical model as a baseline

In [ ]:
import torch
from torch import nn
from torchmetrics import Accuracy
import lightning as L

import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(x)
        return out


class WideResNetBody(nn.Module):
    def __init__(self, depth=16, width_factor=8):
        super().__init__()
        assert (depth - 4) % 6 == 0, "Depth should be 6n+4"
        n = (depth - 4) // 6

        k = width_factor
        self.in_planes = 16

        # Initial conv
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)

        # 3 groups
        self.layer1 = self._make_layer(16*k, n, stride=1)
        self.layer2 = self._make_layer(32*k, n, stride=2)
        self.layer3 = self._make_layer(64*k, n, stride=2)

        self.bn = nn.BatchNorm2d(64*k)

    def _make_layer(self, out_planes, blocks, stride):
        strides = [stride] + [1]*(blocks-1)
        layers = []
        for s in strides:
            layers.append(BasicBlock(self.in_planes, out_planes, s))
            self.in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        return out


class BasicImageClassifier(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            WideResNetBody(),
            nn.LazyLinear(out_features=len(train_data.classes))
        )
        self.loss_fn = nn.CrossEntropyLoss()
        self.metrics = [
            Accuracy(task="multiclass", num_classes=len(train_data.classes))
        ]

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log('val_loss', loss, prog_bar=True)
        for metric in self.metrics:
            metric = metric.to(logits.device)
            result = metric(logits, y)
            if isinstance(result, dict):
                for name, value in result.items():
                    self.log(f"val_{name}", value, on_step=False, on_epoch=True, prog_bar=True)
            else:
                self.log(f"val_{metric.__class__.__name__}", result, on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

basic_model = BasicImageClassifier()
basic_model

BasicImageClassifier(
 (model): Sequential(
 (0): WideResNetBody(
 (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (layer1): Sequential(
 (0): BasicBlock(
 (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv1): Conv2d(16, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (shortcut): Conv2d(16, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
 )
 (1): BasicBlock(
 (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (shortcut): Sequential()
 )
 )
 (layer2): Sequential(
 (0): BasicBlock(
 (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (shortcut): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
 )
 (1): BasicBlock(
 (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (shortcut): Sequential()
 )
 )
 (layer3): Sequential(
 (0): BasicBlock(
 (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (shortcut): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
 )
 (1): BasicBlock(
 (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 (shortcut): Sequential()
 )
 )
 (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 )
 (1): LazyLinear(in_features=0, out_features=100, bias=True)
 )
 (loss_fn): CrossEntropyLoss()
)

# Train the basic model

In [ ]:
from lightning.pytorch.loggers import CSVLogger

basic_logger = CSVLogger(save_dir="lightning_logs", name="basic_model")
basic_trainer = L.Trainer(max_epochs=epochs, accelerator="auto", enable_checkpointing=False, logger=basic_logger)
basic_trainer.fit(basic_model, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


W0514 11:31:24.115000 46152 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/rturnbull/MDAP/hierarchicalsoftmax/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/model_summary/model_summary.py:477: The total number of parameters detected may be inaccurate because the model contains an instance of `UninitializedParameter`. To get an accurate number, set `self.example_input_array` in your LightningModule.

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Sequential       | 11.0 M | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
11.0 M    Trainable params
0         Non-trainable params
11.0 M    Total params
43.825    Total estimated model params size (MB)
45        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                                                                      | 0/? [00:00<?, ?it/s]

/Users/rturnbull/MDAP/hierarchicalsoftmax/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|                                                                                                                     | 0/2 [00:00<?, ?it/s]

...

Epoch 0:  21%|████████████████████▎                                                                            | 327/1563 [00:21<01:23, 14.89it/s, v_num=15, train_loss=4.110]

Epoch 0:  21%|████████████████████▎                                                                            | 327/1563 [00:22<01:23, 14.86it/s, v_num=15, train_loss=4.120]

Epoch 0:  21%|████████████████████▎                                                                            | 328/1563 [00:22<01:22, 14.89it/s, v_num=15, train_loss=4.120]


Detected KeyboardInterrupt, attempting graceful shutdown ...


<span class="codehilite"><div class="highlight"><pre><span></span><span class="gt">Traceback (most recent call last):</span>
  File <span class="nb">&quot;/Users/rturnbull/MDAP/hierarchicalsoftmax/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/call.py&quot;</span>, line <span class="m">48</span>, in <span class="n">_call_and_handle_interrupt</span>
<span class="w">    </span><span class="k">return</span> <span class="n">trainer_fn</span><span class="p">(</span><span class="o">*</span><span class="n">args</span><span class="p">,</span> <span class="o">**</span><span class="n">kwargs</span><span class="p">)</span>
  File <span class="nb">&quot;/Users/rturnbull/MDAP/hierarchicalsoftmax/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/trainer.py&quot;</span>, line <span class="m">599</span>, in <span class="n">_fit_impl</span>
<span class="w">    </span><span class="bp">self</span><span class="o">.</span><span class="n">_run</span><span class="p">(</span><span cla

In [ ]:
import pandas as pd
from pathlib import Path

basic_metrics_df = pd.read_csv(Path(basic_logger.log_dir) / "metrics.csv")
basic_metrics_df = basic_metrics_df.dropna(subset=["val_MulticlassAccuracy"])
basic_fig = go.Figure()
basic_fig.add_trace(go.Scatter(x=basic_metrics_df["epoch"], y=basic_metrics_df["val_MulticlassAccuracy"], mode='lines', name='class'))
basic_fig.update_layout(
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
)
basic_fig.show()


<span class="codehilite"><div class="highlight"><pre><span></span><span class="gt">Traceback (most recent call last):</span>
  File <span class="nb">&quot;/Users/rturnbull/MDAP/hierarchicalsoftmax/.venv/lib/python3.13/site-packages/marimo/_runtime/executor.py&quot;</span>, line <span class="m">122</span>, in <span class="n">execute_cell</span>
<span class="w">    </span><span class="n">exec</span><span class="p">(</span><span class="n">cell</span><span class="o">.</span><span class="n">body</span><span class="p">,</span> <span class="n">glbls</span><span class="p">)</span>
<span class="w">    </span><span class="pm">~~~~^^^^^^^^^^^^^^^^^^</span>
  File <span class="nb">&quot;/var/folders/5x/vn169pd92hb1twlj61zvwpw40000gq/T/marimo_46152/__marimo__cell_emfo_.py&quot;</span>, line <span class="m">5</span>, in <span class="n">&lt;module&gt;</span>
<span class="w">    </span><span class="n">basic_metrics_df</span> <span class="o">=</span> <span class="n">basic_metrics_df</span><span class="

## Hierarchical model

Let's now create a hierarchical model.
First we need to create a tree structure for the CIFAR dataset.

In [ ]:
from hierarchicalsoftmax import (
    SoftmaxNode,
    HierarchicalSoftmaxLazyLinear,
    HierarchicalSoftmaxLoss,
)
from hierarchicalsoftmax.metrics import RankAccuracyTorchMetric

if len(train_data.classes) == 10:
    # CIFAR-10
    superclasses = {
        "animals": ["bird", "cat", "deer", "dog", "frog", "horse"],
        "vehicles": ["airplane", "automobile", "ship", "truck"],
    }
else:
    # CIFAR-100
    superclasses = {
        "aquatic mammals": ["beaver", "dolphin", "otter", "seal", "whale"],
        "fish": ["aquarium_fish", "flatfish", "ray", "shark", "trout"],
        "flowers": ["orchid", "poppy", "rose", "sunflower", "tulip"],
        "food containers": ["bottle", "bowl", "can", "cup", "plate"],
        "fruit and vegetables": ["apple", "mushroom", "orange", "pear", "sweet_pepper"],
        "household electrical devices": ["clock", "keyboard", "lamp", "telephone", "television"],
        "household furniture": ["bed", "chair", "couch", "table", "wardrobe"],
        "insects": ["bee", "beetle", "butterfly", "caterpillar", "cockroach"],
        "large carnivores": ["bear", "leopard", "lion", "tiger", "wolf"],
        "large man-made outdoor things": ["bridge", "castle", "house", "road", "skyscraper"],
        "large natural outdoor scenes": ["cloud", "forest", "mountain", "plain", "sea"],
        "large omnivores and herbivores": ["camel", "cattle", "chimpanzee", "elephant", "kangaroo"],
        "medium-sized mammals": ["fox", "porcupine", "possum", "raccoon", "skunk"],
        "non-insect invertebrates": ["crab", "lobster", "snail", "spider", "worm"],
        "people": ["baby", "boy", "girl", "man", "woman"],
        "reptiles": ["crocodile", "dinosaur", "lizard", "snake", "turtle"],
        "small mammals": ["hamster", "mouse", "rabbit", "shrew", "squirrel"],
        "trees": ["maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree"],
        "vehicles 1": ["bicycle", "bus", "motorcycle", "pickup_truck", "train"],
        "vehicles 2": ["lawn_mower", "rocket", "streetcar", "tank", "tractor"],
    }


root = SoftmaxNode("root")
for superclass, classes in superclasses.items():
    superclass_node = SoftmaxNode(superclass, parent=root)
    for class_name in classes:
        SoftmaxNode(class_name, parent=superclass_node)

# Now that the tree is built, we can set the indexes
# This makes the tree read-only
root.set_indexes()
name_to_node_id = {node.name: root.node_to_id[node] for node in root.leaves}
index_to_node_id = {
    i: name_to_node_id[name] for i, name in enumerate(train_data.classes)
}

# Render the hierarchy
mo.Html(root.svg())

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Title: tree Pages: 1 -->
 
 
 tree 
 
<!-- root -->
 
 root 
 
 root 
 
<!-- aquatic mammals -->
 
 aquatic mammals 
 
 aquatic mammals 
 
<!-- root->aquatic mammals -->
 
 root->aquatic mammals 
 
 
 
<!-- fish -->
 
 fish 
 
 fish 
 
<!-- root->fish -->
 
 root->fish 
 
 
 
<!-- flowers -->
 
 flowers 
 
 flowers 
 
<!-- root->flowers -->
 
 root->flowers 
 
 
 
<!-- food containers -->
 
 food containers 
 
 food containers 
 
<!-- root->food containers -->
 
 root->food containers 
 
 
 
<!-- fruit and vegetables -->
 
 fruit and vegetables 
 
 fruit and vegetables 
 
<!-- root->fruit and vegetables -->
 
 root->fruit and vegetables 
 
 
 
<!-- household electrical devices -->
 
 household electrical devices 
 
 household electrical devices 
 
<!-- root->household electrical devices -->
 
 root->household electrical devices 
 
 
 
<!-- household furniture -->
 
 household furniture 
 
 household furniture 
 
<!-- root->household furniture -->
 
 root->household furniture 
 
 
 
<!-- insects -->
 
 insects 
 
 insects 
 
<!-- root->insects -->
 
 root->insects 
 
 
 
<!-- large carnivores -->
 
 large carnivores 
 
 large carnivores 
 
<!-- root->large carnivores -->
 
 root->large carnivores 
 
 
 
<!-- large man-made outdoor things -->
 
 large man-made outdoor things 
 
 large man-made outdoor things 
 
<!-- root->large man-made outdoor things -->
 
 root->large man-made outdoor things 
 
 
 
<!-- large natural outdoor scenes -->
 
 large natural outdoor scenes 
 
 large natural outdoor scenes 
 
<!-- root->large natural outdoor scenes -->
 
 root->large natural outdoor scenes 
 
 
 
<!-- large omnivores and herbivores -->
 
 large omnivores and herbivores 
 
 large omnivores and herbivores 
 
<!-- root->large omnivores and herbivores -->
 
 root->large omnivores and herbivores 
 
 
 
<!-- medium-sized mammals -->
 
 medium-sized mammals 
 
 medium-sized mammals 
 
<!-- root->medium-sized mammals -->
 
 root->medium-sized mammals 
 
 
 
<!-- non-insect invertebrates -->
 
 non-insect invertebrates 
 
 non-insect invertebrates 
 
<!-- root->non-insect invertebrates -->
 
 root->non-insect invertebrates 
 
 
 
<!-- people -->
 
 people 
 
 people 
 
<!-- root->people -->
 
 root->people 
 
 
 
<!-- reptiles -->
 
 reptiles 
 
 reptiles 
 
<!-- root->reptiles -->
 
 root->reptiles 
 
 
 
<!-- small mammals -->
 
 small mammals 
 
 small mammals 
 
<!-- root->small mammals -->
 
 root->small mammals 
 
 
 
<!-- trees -->
 
 trees 
 
 trees 
 
<!-- root->trees -->
 
 root->trees 
 
 
 
<!-- vehicles 1 -->
 
 vehicles 1 
 
 vehicles 1 
 
<!-- root->vehicles 1 -->
 
 root->vehicles 1 
 
 
 
<!-- vehicles 2 -->
 
 vehicles 2 
 
 vehicles 2 
 
<!-- root->vehicles 2 -->
 
 root->vehicles 2 
 
 
 
<!-- beaver -->
 
 beaver 
 
 beaver 
 
<!-- aquatic mammals->beaver -->
 
 aquatic mammals->beaver 
 
 
 
<!-- dolphin -->
 
 dolphin 
 
 dolphin 
 
<!-- aquatic mammals->dolphin -->
 
 aquatic mammals->dolphin 
 
 
 
<!-- otter -->
 
 otter 
 
 otter 
 
<!-- aquatic mammals->otter -->
 
 aquatic mammals->otter 
 
 
 
<!-- seal -->
 
 seal 
 
 seal 
 
<!-- aquatic mammals->seal -->
 
 aquatic mammals->seal 
 
 
 
<!-- whale -->
 
 whale 
 
 whale 
 
<!-- aquatic mammals->whale -->
 
 aquatic mammals->whale 
 
 
 
<!-- aquarium_fish -->
 
 aquarium_fish 
 
 aquarium_fish 
 
<!-- fish->aquarium_fish -->
 
 fish->aquarium_fish 
 
 
 
<!-- flatfish -->
 
 flatfish 
 
 flatfish 
 
<!-- fish->flatfish -->
 
 fish->flatfish 
 
 
 
<!-- ray -->
 
 ray 
 
 ray 
 
<!-- fish->ray -->
 
 fish->ray 
 
 
 
<!-- shark -->
 
 shark 
 
 shark 
 
<!-- fish->shark -->
 
 fish->shark 
 
 
 
<!-- trout -->
 
 trout 
 
 trout 
 
<!-- fish->trout -->
 
 fish->trout 
 
 
 
<!-- orchid -->
 
 orchid 
 
 orchid 
 
<!-- flowers->orchid -->
 
 fl

### Create DataLoaders with hierarchical labels

In [ ]:
class HierarchicalDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, index_to_node_id):
        self.dataset = dataset
        self.index_to_node_id = index_to_node_id

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return image, self.index_to_node_id[label]

    def __len__(self):
        return len(self.dataset)

hierarchical_train_loader = DataLoader(HierarchicalDataset(train_data, index_to_node_id), batch_size=batch_size, shuffle=True)
hierarchical_test_loader = DataLoader(HierarchicalDataset(test_data, index_to_node_id), batch_size=batch_size, shuffle=False)

### Create the Hierarchical Image Classifier model

In [ ]:
class HierarchicalImageClassifier(BasicImageClassifier):
    # Just overriding the init - keep the rest of the code
    def __init__(self, root: SoftmaxNode):
        super().__init__()
        self.model = nn.Sequential(
            model_body(),
            HierarchicalSoftmaxLazyLinear(root=root)
        )
        self.loss_fn = HierarchicalSoftmaxLoss(root)
        self.metrics = [
            RankAccuracyTorchMetric(
                root,
                {1: "superclass_accuracy", 2: "class_accuracy"},
            ),
        ]
        self.root = root

hierarchical_model = HierarchicalImageClassifier(root)        
hierarchical_model

<span class="codehilite"><div class="highlight"><pre><span></span><span class="gt">Traceback (most recent call last):</span>
  File <span class="nb">&quot;/Users/rturnbull/MDAP/hierarchicalsoftmax/.venv/lib/python3.13/site-packages/marimo/_runtime/executor.py&quot;</span>, line <span class="m">122</span>, in <span class="n">execute_cell</span>
<span class="w">    </span><span class="n">exec</span><span class="p">(</span><span class="n">cell</span><span class="o">.</span><span class="n">body</span><span class="p">,</span> <span class="n">glbls</span><span class="p">)</span>
<span class="w">    </span><span class="pm">~~~~^^^^^^^^^^^^^^^^^^</span>
  File <span class="nb">&quot;/var/folders/5x/vn169pd92hb1twlj61zvwpw40000gq/T/marimo_46152/__marimo__cell_qnkX_.py&quot;</span>, line <span class="m">18</span>, in <span class="n">&lt;module&gt;</span>
<span class="w">    </span><span class="n">hierarchical_model</span> <span class="o">=</span> <span class="n">HierarchicalImageClassifier</span

In [ ]:
hierarchical_logger = CSVLogger(save_dir="lightning_logs", name="hierarchical_model")
hierarchical_trainer = L.Trainer(max_epochs=epochs, accelerator="auto", enable_checkpointing=False, logger=hierarchical_logger)
hierarchical_trainer.fit(hierarchical_model, train_dataloaders=hierarchical_train_loader, val_dataloaders=hierarchical_test_loader)

### Plot the validation results at both the superclass and the class levels

In [ ]:
hierarchical_df = pd.read_csv(Path(hierarchical_logger.log_dir) / "metrics.csv")
hierarchical_df = hierarchical_df.dropna(subset=["val_class_accuracy"])
hierarchical_fig = go.Figure()
hierarchical_fig.add_trace(go.Scatter(x=hierarchical_df["epoch"], y=hierarchical_df["val_superclass_accuracy"], mode='lines', name='superclass'))
hierarchical_fig.add_trace(go.Scatter(x=hierarchical_df["epoch"], y=hierarchical_df["val_class_accuracy"], mode='lines', name='class'))
hierarchical_fig.update_layout(
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
)
hierarchical_fig